# MISA
Alohan'ny mamerina dia avereno atao Run ny notebook iray manontolo. Ny fanaovana azy dia redémarrena mihitsy ny kernel aloha (jereo menubar, safidio **Kernel$\rightarrow$Restart Kernel and Run All Cells**).

Izay misy hoe `YOUR CODE HERE` na `YOUR ANSWER HERE` ihany no fenoina. Afaka manampy cells vaovao raha ilaina. Aza adino ny mameno references eo ambany raha ilaina.

## References
Eto ilay references rehetra no apetraka

---

# Linear Regression

In [1]:
import numpy as np
import scipy
from sklearn.metrics import mean_squared_error
from sklearn import datasets
from sklearn.linear_model import LinearRegression as LR
from sklearn.linear_model import Ridge

def rel_error(x, y):
  """ returns relative error """
  return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))

data = datasets.load_diabetes()
X_train, y_train = data.data, data.target

In [2]:
def fit_inverse(X, y):
    """Direct method using the inverse"""
    # YOUR CODE HERE
    return (np.linalg.inv(X.T.dot(X))).dot(X.T).dot(y)


In [3]:
w = fit_inverse(X_train, y_train)

sk_model = LR(fit_intercept=False)
sk_model.fit(X_train, y_train)

error = rel_error(sk_model.coef_, w)
print("prediction error: ", error)
assert error <= 1e-12

prediction error:  5.534234436929687e-14


In [4]:
def fit_cholesky(X, y):
    """ Cholesky approach """
    # YOUR CODE HERE
    L= np.linalg.cholesky(X.T.dot(X))
    b=(X.T).dot(y)
    w= np.linalg.solve(L, b)
    w1=np.linalg.solve(L.T,w)

    return w1


In [5]:
w = fit_cholesky(X_train, y_train)

sk_model = LR(fit_intercept=False)
sk_model.fit(X_train, y_train)

error = rel_error(sk_model.coef_, w)
print("prediction error: ", error)
assert error <= 1e-12

prediction error:  2.342479867140988e-14


In [6]:
def fit_qr(X, y):
    """ QR approach"""
    # YOUR CODE HERE
    Q, R = np.linalg.qr(X.T.dot(X))
    b = (X.T).dot(y)
    w= (Q.T).dot(b)
    w1=np.linalg.solve(R,w)
    return w1



In [7]:
w = fit_qr(X_train, y_train)

sk_model = LR(fit_intercept=False)
sk_model.fit(X_train, y_train)

error = rel_error(sk_model.coef_, w)
print("prediction error: ", error)
assert error <= 1e-12

prediction error:  4.8591562845214586e-14


In [8]:
def fit_svd(X, y):
    """ SVD approach"""
    # YOUR CODE HERE
    U,S,VT = np.linalg.svd((X.T).dot(X))
    b = (X.T).dot(y)
    w=(U.T).dot(b)
    w1 = (1/S) * w
    w2=(VT.T).dot(w1)

    return w2


In [9]:
w = fit_svd(X_train, y_train)

sk_model = LR(fit_intercept=False)
sk_model.fit(X_train, y_train)

error = rel_error(sk_model.coef_, w)
print("prediction error: ", error)
assert error <= 1e-12

prediction error:  4.148917811674967e-14


## Combine everything in a class

In [10]:
class LinearRegression():
    def __init__(self, fit_intercept=True, method="inverse"):
        self.w = 0
        self.fit_intercept = fit_intercept # bias
        self.method = method

    def fit(self, X, y):
        if self.fit_intercept:
            X = np.hstack((X,np.ones((X.shape[0], 1))))
        if self.method == "inverse":
            self.w = fit_inverse(X, y)
        elif self.method =="cholesky":
            self.w= fit_cholesky(X, y)
        elif self.method =="qr":
            self.w = fit_qr(X, y)
        elif self.method =="svd":
            self.w = fit_svd(X, y)

    def predict(self, X):
        if self.fit_intercept:
            X = np.hstack((X,np.ones((X.shape[0], 1))))
        y=X.dot(self.w)
        return y

### without the bias term

In [11]:
# DIRECT INVERSE APPROACH
sk_model = LR(fit_intercept=False)
sk_model.fit(X_train, y_train)
sk_pred = sk_model.predict(X_train)

model = LinearRegression(fit_intercept=False)
model.fit(X_train, y_train)
pred = model.predict(X_train)

error = rel_error(pred, sk_pred)
print("prediction error: ", error)
assert error <= 1e-11

prediction error:  4.1506436737013384e-13


In [12]:
# OTHER APPROACHES
sk_model = LR(fit_intercept=False)
sk_model.fit(X_train, y_train)
sk_pred = sk_model.predict(X_train)

model_cholesky = LinearRegression(fit_intercept=False, method="cholesky")
model_cholesky.fit(X_train, y_train)
pred_cholesky = model_cholesky.predict(X_train)

error_cholesky = rel_error(pred_cholesky, sk_pred)
print("prediction error cholesky: ", error_cholesky)
assert error_cholesky <= 1e-11

model_qr = LinearRegression(fit_intercept=False, method="qr")
model_qr.fit(X_train, y_train)
pred_qr = model_qr.predict(X_train)

error_qr = rel_error(pred_qr, sk_pred)
print("prediction error qr: ", error_qr)
assert error_qr <= 1e-11

model_svd = LinearRegression(fit_intercept=False, method="svd")
model_svd.fit(X_train, y_train)
pred_svd = model_svd.predict(X_train)

error_svd = rel_error(pred_svd, sk_pred)
print("prediction error svd: ", error_svd)
assert error_svd <= 1e-11

prediction error cholesky:  1.8447305216461452e-13
prediction error qr:  1.1858981924867295e-13
prediction error svd:  4.2824101395367383e-13


### with the bias term

In [13]:
# DIRECT INVERSE APPROACH
sk_model = LR(fit_intercept=True)
sk_model.fit(X_train, y_train)
sk_pred = sk_model.predict(X_train)

model = LinearRegression(fit_intercept=True)
model.fit(X_train, y_train)
pred = model.predict(X_train)

error = rel_error(pred, sk_pred)
print("prediction error: ", error)
assert error <= 1e-13

prediction error:  1.1709395221326805e-14


In [14]:
# OTHER APPROACHES
sk_model = LR(fit_intercept=True)
sk_model.fit(X_train, y_train)
sk_pred = sk_model.predict(X_train)

model_cholesky = LinearRegression(fit_intercept=True, method="cholesky")
model_cholesky.fit(X_train, y_train)
pred_cholesky = model_cholesky.predict(X_train)

error_cholesky = rel_error(pred_cholesky, sk_pred)
print("prediction error cholesky: ", error_cholesky)
assert error_cholesky <= 1e-11

model_qr = LinearRegression(fit_intercept=True, method="qr")
model_qr.fit(X_train, y_train)
pred_qr = model_qr.predict(X_train)

error_qr = rel_error(pred_qr, sk_pred)
print("prediction error qr: ", error_qr)
assert error_qr <= 1e-11

model_svd = LinearRegression(fit_intercept=True, method="svd")
model_svd.fit(X_train, y_train)
pred_svd = model_svd.predict(X_train)

error_svd = rel_error(pred_svd, sk_pred)
print("prediction error svd: ", error_svd)
assert error_svd <= 1e-11

prediction error cholesky:  2.8651623773297924e-15
prediction error qr:  8.020117523603142e-15
prediction error svd:  1.3363753966653326e-13


## Compare

Compare the running time of the different approaches on a large dataset

# Ridge Regression

In [15]:
def fit_inverse_ridge(X, y, alpha=1.0, fit_intercept=False):
    """Direct method using the inverse"""
    # YOUR CODE HERE
    
    if fit_intercept:
        X = np.hstack((X,np.ones((X.shape[0], 1))))
    theta = (np.linalg.inv(X.T.dot(X)+ alpha * np.identity(X.shape[1]))).dot(X.T).dot(y)
    return theta

In [16]:
w = fit_inverse_ridge(X_train, y_train, alpha=0.1)

sk_model = Ridge(fit_intercept=False, alpha=0.1)
sk_model.fit(X_train, y_train)

error = rel_error(sk_model.coef_, w)
print("prediction error: ", error)
assert error <= 1e-11

prediction error:  5.946841239031182e-14


In [17]:
class RidgeRegression(LinearRegression):
    def __init__(self, fit_intercept=True, method="inverse", alpha=1.0):
        super(RidgeRegression, self).__init__(fit_intercept, method)
        self.alpha = alpha

    def fit(self, X, y):
        A=np.sqrt(self.alpha*np.eye(X.shape[1]))
        X_tilde = np.vstack((X,A))
        y_tilde = np.hstack((y,np.zeros(X.shape[1])))

        super(RidgeRegression, self).fit(X_tilde, y_tilde)

### without bias

In [18]:
# OTHER APPROACHES
sk_model = Ridge(fit_intercept=False, alpha=0.1)
sk_model.fit(X_train, y_train)
sk_pred = sk_model.predict(X_train)

model = RidgeRegression(fit_intercept=False, method="inverse", alpha=0.1)
model.fit(X_train, y_train)
pred = model.predict(X_train)

error = rel_error(pred, sk_pred)
print("prediction error inverse: ", error)
assert error <= 1e-11

model_qr = RidgeRegression(fit_intercept=False, method="qr", alpha=0.1)
model_qr.fit(X_train, y_train)
pred_qr = model_qr.predict(X_train)

error_qr = rel_error(pred_qr, sk_pred)
print("prediction error qr: ", error_qr)
assert error_qr <= 1e-11

model_svd = RidgeRegression(fit_intercept=False, method="svd", alpha=0.1)
model_svd.fit(X_train, y_train)
pred_svd = model_svd.predict(X_train)

error_svd = rel_error(pred_svd, sk_pred)
print("prediction error svd: ", error_svd)
assert error_svd <= 1e-11

prediction error inverse:  4.894378670210236e-14
prediction error qr:  3.02249108444111e-14
prediction error svd:  7.771630233113505e-14
